![alt text](https://anvaqta.id/headerai.jpg)

# Visualisasi Bobot Klasifikasi Linier

## *Deep Representations*

Seperti yang telah dijelaskan sebelumnya, Salah satu intuisi alasan mengapa Jaringan yang dalam mampu mengatasi permasalahan lebih baik adalah dikarenakan Jaringan tersebut mampu membuat representasi permasalahan dengan lebih detil dan lebih baik

Sebagai contoh, misalkan kita membangun Jaringan Saraf Tiruan untuk mengenali identitas seseorang berdasarkan citra wajah. 

<img src="https://image.ibb.co/iOG8MK/deep_representation.jpg" alt="deep_representation"/>

Layer pertama pada Jaringan yang telah dilatih dengan ratusan dan ribuan citra mungkin akan bertindak lebih sebagai suatu pengenalan fitur atau *Edge Detection*, di mana setiap neuron mencoba mengenali orientasi garis yang berbeda. Kemudian terdapat layer berikutnya yang dilatih terhadap data garis hasil ekstraksi layer pertama dan menggabungkannya, dan mungkin membentuk suatu pengenalan fitur baru misalnya membentuk neuron yang mengenali fitur mata, hidung, dan lain sebagainya. Dan hal ini terus dilakukan untuk layer-layer berikutnya sehingga Jaringan dengan layer yang banyak dapat membentuk representasi fitur yang lebih detil.

Namun apakah benar demikian?
Mari kita selidiki

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

---
## Load MNIST Dataset

In [0]:
!pip install tensorflow-gpu==2.0.0

In [0]:
import tensorflow as tf
tf.__version__

In [0]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

print('X_train.shape =',X_train.shape)
print('y_train.shape =',y_train.shape)
print('X_test.shape  =',X_test.shape)
print('y_test.shape  =',y_test.shape)

Tampilkan 20 gambar pertama


In [0]:
fig, ax = plt.subplots(2,10,figsize=(15,4.5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
for j in range(0,2):
    for i in range(0, 10):
        ax[j,i].imshow(X_train[i+j*10], cmap='gray')
        ax[j,i].set_title(y_train[i+j*10])
        ax[j,i].axis('off')
plt.show()

Sebelumnya kita sudah mengklasifikasi Mnist Digit, pada tahap awal jaringan kita melakukan Flatten, mari kita lakukan menggunakan numpy


In [0]:
# reshape X_train menjadi 2 dimensi berukuran 60000 x (28*28) menggunakan np.reshape
X_train_reshape = np.reshape(X_train, (60000, (28*28)))

# reshape X_test menjadi 2 dimensi berukuran 10000 x (28*28) menggunakan np.reshape
X_test_reshape = np.reshape(X_test, (10000, (28*28)))

# reshape y_train menjadi 1 dimensi dengan fungsi np.ravel() 
y_train_reshape = np.ravel(y_train)
# reshape y_test menjadi 1 dimensi dengan fungsi np.ravel()
y_test_reshape = np.ravel(y_test)

print('X_train:', X_train_reshape.shape)
print('X_test:', X_test_reshape.shape)

print('y_train:', y_train_reshape.shape)
print('y_test:', y_test_reshape.shape)

---
## Latih model menggunakan Scikit-Learn

In [0]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', max_iter=2000, verbose=2, n_jobs=2,learning_rate='invscaling', eta0=1e-7)
clf.fit(X_train_reshape, y_train_reshape)

Hitung akurasinya

In [0]:
y_pred = clf.predict(X_test_reshape)

from sklearn.metrics import accuracy_score
print('akurasi =',accuracy_score(y_test_reshape, y_pred)*100,'%')

---
## Ambil nilai bobot
Untuk memvisualisasikan bobot dari jaringan, kita ambil nilai bobotnya terlebih dahulu

In [0]:
from sklearn.preprocessing import MinMaxScaler
weights = clf.coef_
class_id = 0
w0 = weights[class_id]
scaler = MinMaxScaler((0.1,.9))
w0 = scaler.fit_transform(w0.reshape(-1, 1))
print(w0.shape)

Visualisaikan bobot

In [0]:
fig, ax = plt.subplots(1,10,figsize=(18,5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
for i in range(0, 10):
    w = weights[i]
    
    w = scaler.fit_transform(w.reshape(-1, 1))
    
    w = w.reshape((28,28))
    
    ax[i].imshow(w, cmap='gray')
    ax[i].axis('off')
    ax[i].set_title(i)
plt.show()

---
# Visualisasi Bobot Cifar-10

---
## Load Cifar-10

In [0]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()


classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print('X_train.shape =',X_train.shape)
print('y_train.shape =',y_train.shape)
print('X_test.shape  =',X_test.shape)
print('y_test.shape  =',y_test.shape)

Tampilkan 20 data pertama beserta kelasnya dari 50000 data latih

In [0]:
fig, ax = plt.subplots(2,10,figsize=(15,4.5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
for j in range(0,2):
    for i in range(0, 10):
        ax[j,i].imshow(X_train[i+j*10])
        ax[j,i].set_title(classes[y_train[i+j*10,0]])
        ax[j,i].axis('off')
plt.show()

Selanjutnya kita akan mengklasifikasikan 10 kelas data cifar-10 menggunakan Klasifikasi Linier

Untuk itu, pertama kali kita preprocessing data dengan melakukan *zero-mean centering*

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

mean_image = np.mean(X_train, axis = 0)
X_train -= mean_image
X_test -= mean_image

Pada Klasifikasi Linier, input harus diubah menjadi satu dimensi, 


<img src="https://i.ibb.co/HDTHkp9/linear-algebra.png" alt="klasifikasi linier"/>


karena itu, lakukan reshape

In [0]:
# reshape X_train menjadi 2 dimensi berukuran 50000 x (32*32*3)
X_train_reshape = np.reshape(50000, (32*32*3))

# reshape X_test menjadi 2 dimensi berukuran 10000 x (32*32*3)
X_test_reshape = np.reshape(10000, (32*32*3))

# reshape y_train menjadi 1 dimensi dengan fungsi ravel()
y_train_reshape = np.ravel(y_train)
# reshape y_test menjadi 1 dimensi dengan fungsi ravel()
y_test_reshape = np.ravel(y_test)

print('X_train:', X_train_reshape.shape)
print('X_test:', X_test_reshape.shape)

print('y_train:', y_train_reshape.shape)
print('y_test:', y_test_reshape.shape)

**EXPECTED OUTPUT**
<pre>
 X_train: (50000, 3072)
 X_test: (10000, 3072)
 y_train: (50000,)
 y_test: (10000,)

---
## Klasifikasi dengan Sci-Kit Learn
Selanjutnya kita lakukan pelatihan Klasifikasi Linier menggunakan library sklearn SGDClassifier

In [0]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', max_iter=2000, verbose=2, n_jobs=2,learning_rate='invscaling', eta0=1e-7)
clf.fit(X_train_reshape, y_train_reshape)

Kita bisa mengecek akurasi dari model yang telah dilatih. 

Dapat dilihat bahwa akurasi masih kurang bagus dikarenakan kita hanya menggunakan klasifikasi linier atau Jaringan Saraf Tiruan satu layer

In [0]:
y_pred = clf.predict(X_test_reshape)

from sklearn.metrics import accuracy_score
print('akurasi =',accuracy_score(y_test_reshape, y_pred)*100,'%')

---
## Visualisasi Bobot

Selanjutnya mari kita ambil bobot yang telah dilatih dari model.

Perhatikan bahwa ukuran bobot adalah (10 x 3072)

In [0]:
from sklearn.preprocessing import MinMaxScaler
weights = clf.coef_
class_id = 0
w0 = weights[class_id]
scaler = MinMaxScaler((0.1,.9))
w0 = scaler.fit_transform(w0.reshape(-1, 1))
print(w0.shape)

Sekarang, mari kita tampilkan visualisasi keseluruhan bobot

In [0]:
fig, ax = plt.subplots(1,10,figsize=(18,5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
for i in range(0, 10):
    w = weights[i]
    
    w = scaler.fit_transform(w.reshape(-1, 1))
    
    w = w.reshape((32,32,3))
    
    ax[i].imshow(w)
    ax[i].axis('off')
    ax[i].set_title(classes[i])
plt.show()